In [5]:
from htbac import Runner, Protocol, Simulation, System, AbFile, DataAggregate
from htbac.protocols import RFE, ESMACS # Allows users to utilize pre-defined protocols 

2018-08-29 12:14:16,615: htbac.abpath        : MainProcess                     : MainThread     : INFO    : python.interpreter   version: 2.7.12 (default, Oct 11 2016, 05:24:00) [GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.38)]
2018-08-29 12:14:16,615: htbac.abpath        : MainProcess                     : MainThread     : INFO    :                      pid/tid: 30468/MainThread
2018-08-29 12:14:16,616: htbac.simulation    : MainProcess                     : MainThread     : INFO    : python.interpreter   version: 2.7.12 (default, Oct 11 2016, 05:24:00) [GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.38)]
2018-08-29 12:14:16,616: htbac.simulation    : MainProcess                     : MainThread     : INFO    :                      pid/tid: 30468/MainThread
2018-08-29 12:14:16,617: htbac.htbac         : MainProcess                     : MainThread     : INFO    : python.interpreter   version: 2.7.12 (default, Oct 11 2016, 05:24:00) [GCC 4.2.1 Compatible Apple LLVM 8.0.0 (cl

In [6]:
# define system
pdb = AbFile('systems/ptp1b-l1-l2-complex.pdb', tag='pdb')
top = AbFile('systems/ptp1b-l1-l2-complex.top', tag='topology')
tag = AbFile('systems/ptp1b-l1-l2-tags.pdb', tag='alchemicaltags')
cor = AbFile('systems/ptp1b-l1-l2-complex.inpcrd', tag='coordinate')
system = System(name='ptp1b-l1-l2', files=[pdb, top, tag, cor])

In [7]:
import os 
print os.path.exists("/Users/JumanaDakka/git/htbac/examples/default-configs/rfe/ties-0.conf")
print(os.getcwd())

False
/Users/JumanaDakka/git/htbac/examples


In [4]:
# Create simulation steps for relative free energy protocol

# define protocol:
p = Protocol()

# define step 0: 
s0 = Simulation(name='minimizer')
s0.engine = 'openmm'
s0.add_ensemble('replica', range(5))
s0.add_ensemble('lambdawindow', [1.0, 0.5, 0.0])
s0.numsteps = 5000 
s0.add_input_file("default_configs/rfe/ties-0.conf", is_executable_argument=True)
s0.system = system


# append step 0 to protocol:
p.append(s0)

# define step 1: 
s1 = Simulation(name='equilibrate')
s1.engine = 'openmm'
s1.add_ensemble('replica', range(5))
s1.add_ensemble('lambdawindow', [1.0, 0.5, 0.0])
s1.add_input_file("default_configs/rfe/ties-1.conf", is_executable_argument=True)
s1.numsteps = 50000 
s1.system = system

# append step 1 to protocol:
p.append(s1)

ht = Runner('titan_orte', comm_server=('two.radical-project.org', 33048))
ht.add_protocol(p)
ht.run(walltime=480)


NameError: name 'Protocol' is not defined

In [6]:
# using a pre-defined protocol (RFE), add details to the simulation steps

p = RFE()
p.append(DataAggregate(extension=".alch"))

    # Protocol `p` is made up of 2 steps:
    # minimize -> simulate 

for sim, numsteps in zip(p.simulations(), [5000, 50000]):

    sim.system = system
    sim.engine = 'namd'
    
    sim.cutoff = 12.0
    sim.switchdist = 10.0
    sim.pairlistdist = 13.5
    sim.numsteps = numsteps

    sim.processes = 1
    sim.threads_per_process = 16
    sim.variables = dict()
    sim.add_ensemble('replica', range(5))
    sim.add_ensemble('lambdawindow', [1.0, 0.5, 0.0])
    
ht = Runner('bw_aprun', comm_server=('two.radical-project.org', 33048))
ht.add_protocol(p)
ht.run(walltime=480, queue='high')


2018-08-29 10:55:22,984: htbac.simulation    : MainProcess                     : MainThread     : DEBUG   : simulation: adding var cutoff. Its value is None.
2018-08-29 10:55:22,984: htbac.simulation    : MainProcess                     : MainThread     : DEBUG   : simulation: adding var numsteps. Its value is None.
2018-08-29 10:55:22,984: htbac.simulation    : MainProcess                     : MainThread     : DEBUG   : simulation: adding var switchdist. Its value is None.
2018-08-29 10:55:22,984: htbac.simulation    : MainProcess                     : MainThread     : DEBUG   : simulation: adding var box_z. Its value is None.
2018-08-29 10:55:22,985: htbac.simulation    : MainProcess                     : MainThread     : DEBUG   : simulation: adding var alchemicaltags. Its value is None.
2018-08-29 10:55:22,985: htbac.simulation    : MainProcess                     : MainThread     : DEBUG   : simulation: adding var box_y. Its value is None.
2018-08-29 10:55:22,985: htbac.simulatio

In [ ]:
# ESMACS protocol (4) steps: minimize -> thermal_equilibration -> equilibrate -> production MD

p = ESMACS()
